In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from rdkit.Chem import PandasTools, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn import pipeline as pl
from sklearn import preprocessing as pp
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
#import graphviz 

### Dataset initiation
Initiate and prepare the dataset with descriptors to make them usable for the ML-model. 

In [19]:
dataset_molecules = pd.read_csv('combi.csv')
PandasTools.AddMoleculeColumnToFrame(dataset_molecules, smilesCol='SMILES')
dataset_molecules.head()

,SMILES,ALDH1_inhibition,ROMol
0,[NH3+]CCSSCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x00000203918...
1,[NH3+]CCC[NH2+]CCCC[NH2+]CCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x00000203918...
2,[NH3+]CCCCCCCCCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x00000203918...
3,[NH3+]CCSSCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x00000203918...
4,ClCC[NH+](CCCl)CCCl,0,<rdkit.Chem.rdchem.Mol object at 0x00000203918...


In [35]:
descriptors_labels = [n[0] for n in Descriptors._descList[:]]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors_labels)
rdkit_desc = [calc.CalcDescriptors(m) for m in dataset_molecules["ROMol"]]
dataset_descriptors = pd.DataFrame(rdkit_desc, columns=[descriptors_labels])
dataset_descriptors = dataset_descriptors[['MaxAbsEStateIndex',
 'MinAbsEStateIndex',
 'MinEStateIndex',
 'BCUT2D_CHGHI',
 'BCUT2D_CHGLO',
 'BCUT2D_LOGPLOW',
 'BCUT2D_MRLOW',
 'SMR_VSA1',
 'SMR_VSA5',
 'SlogP_VSA2',
 'EState_VSA10',
 'VSA_EState2',
 'FractionCSP3',
 'NumAliphaticRings']]
X_train, X_test, y_train, y_test = train_test_split(dataset_descriptors, dataset_molecules['ALDH1_inhibition'], test_size=0.2, random_state=42)
dataset_descriptors.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPLOW,BCUT2D_MRLOW,SMR_VSA1,SMR_VSA5,SlogP_VSA2,EState_VSA10,VSA_EState2,FractionCSP3,NumAliphaticRings
0,3.733529,1.036517,1.036517,1.865112,-1.907393,-2.514714,-0.360469,11.467335,0.000000,24.595220,0.0,0.000000,1.0,0
1,3.831341,1.078701,1.078701,1.940948,-2.003261,-2.860321,-0.666457,22.100912,25.683286,39.268538,0.0,0.000000,1.0,0
2,3.837048,1.112603,1.112603,1.908706,-1.984755,-2.495713,-0.368558,11.467335,51.366573,13.089513,0.0,0.000000,1.0,0
3,3.733529,1.036517,1.036517,1.865112,-1.907393,-2.514714,-0.360469,11.467335,0.000000,24.595220,0.0,0.000000,1.0,0
4,5.560741,0.683642,0.683642,2.035557,-2.143624,-3.126526,-0.892277,4.899910,0.000000,37.274280,0.0,1.388889,1.0,0


### Introducing a Pipeline with minmax scaler and Desciscion tree.

In [36]:
scaler = pp.MinMaxScaler()
pca = PCA()
dtree = tree.DecisionTreeClassifier(max_features=None, max_depth=None, max_leaf_nodes=None)

pipeline = pl.Pipeline(steps=[("sc", scaler), ("pca", pca), ("dtreeCLF", dtree)])
cross_score = cross_val_score(pipeline, X_train, y_train, cv=10)
print("Cross score:") 
print(cross_score)
pipeline.fit(X_train, y_train)
print("score:")
print(pipeline.score(X_test, y_test))

Cross score:
[0.675   0.7     0.7125  0.7     0.6875  0.69375 0.68125 0.66875 0.73125
 0.68125]
score:
0.725


### Creating a physical tree

In [37]:
#dot_data = tree.export_graphviz(dtree, out_file=None, feature_names=dataset_descriptors.columns, class_names=['1', '0'])
#graph = graphviz.Source(dot_data) 
#graph.render("descision_tree") 

### Analysing the model

In [38]:
y_true = y_test
y_pred = pipeline.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp/(tp+fp) 
accuracy = (tp+tn)/(tp+tn+fp+fn)
print(" |+\t|-\t|\n+|{:.0f}\t|{:.0f}\t|\n-|{:.0f}\t|{:.0f}\t|".format(tp, fp, fn, tn))
print("Specificiteit: {:.2f} Sensitivity: {:.2f} Accuracy {:.2f}".format(specificity, sensitivity, accuracy))

 |+	|-	|
+|54	|62	|
-|48	|236	|
Specificiteit: 0.79 Sensitivity: 0.47 Accuracy 0.72


This code calculates further on X_test
A selection of 100 molecules is made, based on the selection from pipeline + further reduction by similarity analysis of the selected molecules with the known ALDH1-positives (and negatives)
Can be appended to Constantijns pipeline.
Als er problemen zijn laat maar weten.

## Further processing of the selection

Create new datafame from X_test. Append values for 'ALDH1_inhibition' and 'predicted'

In [39]:
pipeline_results = X_test.copy()
pipeline_results['ALDH1_inhibition'] = y_test
pipeline_results['predicted'] = y_pred
pipeline_results.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPLOW,BCUT2D_MRLOW,SMR_VSA1,SMR_VSA5,SlogP_VSA2,EState_VSA10,VSA_EState2,FractionCSP3,NumAliphaticRings,ALDH1_inhibition,predicted
1860,12.938670,0.088947,-3.642991,2.351664,-2.307988,-2.449569,-0.125741,13.212334,38.129358,41.917069,13.212334,13.750281,0.500000,1,0,1
353,4.597214,0.819103,0.819103,2.225038,-2.049995,-2.684847,-0.460778,5.101408,0.000000,16.531283,0.000000,4.518570,0.000000,1,0,0
1333,12.173495,0.014973,-0.504740,2.132749,-2.018149,-2.195873,0.097039,9.211688,13.468494,10.350345,9.589074,24.034309,0.125000,0,0,1
905,12.442508,0.089206,-0.214087,2.206277,-2.106670,-2.313198,0.086127,14.268263,25.869347,36.336766,4.794537,12.870177,0.411765,1,1,0
1289,5.116451,0.451150,0.451150,1.975855,-2.136225,-2.479488,0.195439,4.736863,6.420822,30.486310,0.000000,0.000000,0.333333,0,1,1


Select only the predicted positives

In [40]:
pipeline_positives = pipeline_results[pipeline_results['predicted'].values == 1]
print("{} positives from pipeline".format(pipeline_positives.shape[0]))

116 positives from pipeline


## Further Selection with Tanamoto similarity

Load all known molecules in frame

In [41]:
from rdkit import DataStructs
tested_molecules = dataset_molecules.copy()
PandasTools.AddMoleculeColumnToFrame(tested_molecules, smilesCol='SMILES')

Divide molecules in Inhibitors and non-Inhibitors

In [42]:
tested_molecules_pos = tested_molecules[tested_molecules['ALDH1_inhibition'] == 1]
tested_molecules_neg = tested_molecules[tested_molecules['ALDH1_inhibition'] == 0]
print("{}% inhibitors present in known molecules".format(100*len(tested_molecules_pos)/len(tested_molecules)))

30.0% inhibitors present in known molecules


Convert both sets to Morgan Fingerprints (to compare with)

In [43]:
from rdkit.Chem import AllChem
bulk_pos = [AllChem.GetMorganFingerprintAsBitVect(x,2) for x in tested_molecules_pos['ROMol']]
bulk_neg = [AllChem.GetMorganFingerprintAsBitVect(x,2) for x in tested_molecules_neg['ROMol']]
len_pos = len(bulk_pos); len_neg = len(bulk_neg)

Get indices from pipeline prediction and select these indices from dataset_molecules.
This must be done to retreive SMILES again, since these are needed for the comparison in the for loop

In [44]:
index_pipeline_positives = pipeline_positives.index
pipeline_positives_mol = dataset_molecules.iloc[index_pipeline_positives]
print("{} positive molecules from pipeline".format(len(pipeline_positives_mol)))

116 positive molecules from pipeline


Calculate similarity with tested positives/negatives for every mol from pipeline_positives

In [45]:
pipeline_positives_mol_copy = pipeline_positives_mol.copy(deep=False)
# Loop all rows from test set
for index, row in pipeline_positives_mol.iterrows():     
    test_smiles = row['SMILES']
    # Make allchem MolFrom the SMILES and convert it into to GetMorganFingerprintAsBitVec
    test_mol = AllChem.MolFromSmiles(test_smiles)
    ref_fps = AllChem.GetMorganFingerprintAsBitVect(test_mol,2)
    # Calculate similarities with positives and negative
    similarity_pos = [DataStructs.FingerprintSimilarity(ref_fps,x) for x in bulk_pos]
    similarity_neg = [DataStructs.FingerprintSimilarity(ref_fps,x) for x in bulk_neg]
    # Add all positive and negative similarities and normalize
    rel_sum_pos = sum(similarity_pos)/len_pos # divide by sizes to normalize the result
    rel_sum_neg = sum(similarity_neg)/len_neg # divide by sizes to normalize the result
    
    pipeline_positives_mol_copy.loc[index, ['SIM_WITH_POSITIVE_SET']] = rel_sum_pos
    pipeline_positives_mol_copy.loc[index, ['SIM_WITH_NEGATIVE_SET']] = rel_sum_neg   

pipeline_positives_mol_copy.head()

,SMILES,ALDH1_inhibition,ROMol,SIM_WITH_POSITIVE_SET,SIM_WITH_NEGATIVE_SET
1860,Cc1n[nH]c(C)c1S(=O)(=O)N1CCCC(C(=O)NCc2cccs2)C1,0,<rdkit.Chem.rdchem.Mol object at 0x00000203918...,0.147624,0.116059
1333,Cc1ccc2c(c1)oc(=O)n2CC(=O)c1ccccc1,0,<rdkit.Chem.rdchem.Mol object at 0x00000203918...,0.148996,0.125943
1289,COCCCNC(=S)NNC(=S)Nc1ccccc1,1,<rdkit.Chem.rdchem.Mol object at 0x00000203918...,0.122796,0.116723
938,COC(=O)c1c(-c2ccco2)csc1NC(=O)C1CC=CCC1C(=O)[O-],1,<rdkit.Chem.rdchem.Mol object at 0x00000203918...,0.134356,0.092620
56,N#CC(Cc1ccccc1OC(F)F)c1nc2ccccc2[nH]1,0,<rdkit.Chem.rdchem.Mol object at 0x00000203918...,0.075265,0.106971


In [46]:
false_positives = pipeline_positives_mol_copy[pipeline_positives_mol_copy['ALDH1_inhibition'] == 0]
print("{:d} predicted positives from pipeline. {:.1f}% false. ".format(len(pipeline_positives_mol_copy),100*len(false_positives)/len(pipeline_positives_mol_copy)))

116 predicted positives from pipeline. 53.4% false. 


Only select molecules with larger similarity with positives than with negatives

In [47]:
selection = pipeline_positives_mol_copy[pipeline_positives_mol_copy['SIM_WITH_POSITIVE_SET'] > pipeline_positives_mol_copy['SIM_WITH_NEGATIVE_SET']]
false_positives = selection[selection['ALDH1_inhibition']==0]
print("{:d} predicted positives after first selection. {:.1f}% false positive. ".format(len(selection),100*len(false_positives)/len(selection)))

102 predicted positives after first selection. 50.0% false positive. 


Sort the selection by similarity with positives and pick the first 100 molecules

In [33]:
selection_copy = selection.copy()
selection_copy = selection_copy.sort_values(by=['SIM_WITH_POSITIVE_SET'], ascending=False);
selection_copy = selection_copy.iloc[0:100]
false_positives = selection_copy[selection_copy['ALDH1_inhibition']==0]
print("{:d} predicted positives after second selection. {:.1f}% false positive. ".format(len(selection_copy),100*len(false_positives)/len(selection_copy)))

100 predicted positives after second selection. 43.0% false positive. 


Try if low score on similarity with negatives works better...

In [34]:
selection_copy = selection.copy()
selection_copy = selection_copy.sort_values(by=['SIM_WITH_NEGATIVE_SET'], ascending=True);
selection_copy = selection_copy.iloc[0:100]
false_positives = selection_copy[selection_copy['ALDH1_inhibition']==0]
print("{:d} predicted positives after second selection. {:.1f}% false positive. ".format(len(selection_copy),100*len(false_positives)/len(selection_copy)))

100 predicted positives after second selection. 43.0% false positive. 
